# *Airline Passenger Satisfaction*
![alt text](https://www.royalcoachman.com/blog/wp-content/uploads/2014/06/happy-passenger-.jpg)

## How can we improve airline passenger satisfaction?

# Imports
Pandas is used for Data Frames and Scipy/Stats is only used to run Pearson correlation test.


In [0]:
import pandas as pd
from scipy import stats

# Setting up the Data Frame
Data Source: https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction

Line 3 combines the two data frames.

Line 4 drops all entries with nulls, was 393 entries out of 129,880.

Lines 5 and 6 quantify the entries in the 'Satisfaction' Column, with 1 being if a customer was satisfied with their experience and 0 being if a customer was neutral or dissatisfied with their experience.

Lines 7 and 8 quantify the entires in the 'Customer' Column, with 1 being a loyal customer and 0 being a disloyal/new customer.



In [0]:
testcsv = pd.read_csv('test.csv')
traincsv = pd.read_csv('train.csv')
airPassSatisDF = pd.concat([testcsv, traincsv])
airPassSatisDF = airPassSatisDF.dropna(how = 'any')
airPassSatisDF = airPassSatisDF.replace('neutral or dissatisfied', 0)
airPassSatisDF = airPassSatisDF.replace('satisfied', 1)
airPassSatisDF = airPassSatisDF.replace('disloyal Customer', 0)
airPassSatisDF = airPassSatisDF.replace('Loyal Customer', 1)
airPassSatisDF.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,19556,Female,1,52,Business travel,Eco,160,5,4,3,4,3,4,3,5,5,5,5,2,5,5,50,44.0,1
1,1,90035,Female,1,36,Business travel,Business,2863,1,1,3,1,5,4,5,4,4,4,4,3,4,5,0,0.0,1
2,2,12360,Male,0,20,Business travel,Eco,192,2,0,2,4,2,2,2,2,4,1,3,2,2,2,0,0.0,0
3,3,77959,Male,1,44,Business travel,Business,3377,0,0,0,2,3,4,4,1,1,1,1,3,1,4,0,6.0,1
4,4,36875,Female,1,49,Business travel,Eco,1182,2,3,4,3,4,1,2,2,2,2,2,4,2,4,0,20.0,1


# Correlation
Ran a correlation test to see which variables in the data have the most correlation to if a passenger is overall satsified or not.

In [0]:
airPassSatisDF.corr()['satisfaction'].sort_values(ascending = False)

satisfaction                         1.000000
Online boarding                      0.501620
Inflight entertainment               0.398334
Seat comfort                         0.348576
On-board service                     0.322329
Leg room service                     0.312557
Cleanliness                          0.306891
Flight Distance                      0.298206
Inflight wifi service                0.283291
Baggage handling                     0.248651
Inflight service                     0.245027
Checkin service                      0.237146
Food and drink                       0.211164
Customer Type                        0.185925
Ease of Online booking               0.168704
Age                                  0.134001
id                                   0.012990
Gate location                       -0.002923
Unnamed: 0                          -0.005672
Departure Delay in Minutes          -0.051032
Departure/Arrival time convenient   -0.054457
Arrival Delay in Minutes          

![picture](https://github.com/andrewcsmith1997/airlinePassengerSatisfaction/raw/master/Images/Correlation.png)

As we can see from the test results and this graph, 'Online Boarding', has the most correlation.

# A/B Test
Conducted A/B tests to see if there was an increase in overall satisfaction between passengers who gave an above average score for a variable and passengers who gave a below average score. This formulates our hypotheses:

###Ho: p1 - p2 ≯  0

###Ha: p1 - p2 > 0

In lines 6 and 7, you can see our null hypothesis is: The difference between the proportions of the two samples is NOT greater than 0.

### In the 'A/B Test Results' Section
In line 9, we get the p-value is obtained using stats and the pearsonr method.

In line 10, the standard error is multiplied by 1.645 because this is a one tail test and being calculated at 95% confidence.

In line 12, we only have a lower limit because this is a one tail test and we're only testing to see if the lower limit is greater than zero.

In lines 14 through 18, the final verdict is determined by the p-value and lower limit. Do we reject or fail to reject the null?



In [0]:
def abtestfunction(columnName, satisfactionColumn):
    '''
    This will calculate the essential statistics of an A/B test.
    Parameter for this function must follow this format: i.e. dataFrameName['ColumnName'].values
    '''
    print('\nHo: p1 - p2 ≯  0',
      '\nHa: p1 - p2 > 0')
    meanScore = 0
    totalScore = 0
    populationSize = 0
    aboveMeanSampleSize = 0
    belowMeanSampleSize = 0
    satisfiedAboveSample = 0
    satisfiedBelowSample = 0
    counter = 0
    # Gets mean and population Size
    for score in columnName:
        totalScore += columnName[populationSize]
        populationSize += 1
    meanScore = (totalScore/populationSize)

    # Gets sample sizes, converted per sample, and conversion rate
    for score in columnName:
        if columnName[counter] > meanScore:
            aboveMeanSampleSize += 1
            if satisfactionColumn[counter] == 1:
                satisfiedAboveSample += 1
        else:
            belowMeanSampleSize += 1
            if satisfactionColumn[counter] == 1:
                satisfiedBelowSample += 1
        counter += 1
    # A/B Test Results
    conversionAboveMean = satisfiedAboveSample / aboveMeanSampleSize
    conversionBelowMean = satisfiedBelowSample / belowMeanSampleSize
    stDevp1 = ((conversionAboveMean * (1 - conversionAboveMean)) / aboveMeanSampleSize) ** 0.5
    stDevp2 = ((conversionBelowMean * (1 - conversionBelowMean)) / belowMeanSampleSize) ** 0.5
    proportionDifference = conversionAboveMean - conversionBelowMean
    pooledProportion = ((conversionAboveMean * aboveMeanSampleSize) + (conversionBelowMean * belowMeanSampleSize)) / populationSize
    standardError = (stDevp1 + stDevp2) ** 0.5
    zStatistic = (conversionAboveMean - conversionBelowMean) / standardError
    pValue = stats.pearsonr(airPassSatisDF['On-board service'], airPassSatisDF['satisfaction'])[1]
    confidenceInterval = standardError * 1.645
    lowerLimit = proportionDifference - confidenceInterval
    finalVerdict = ''
    if (pValue < 0.05) & (lowerLimit > 0):
        finalVerdict = 'We reject the null. We can say with 95% confidence that the difference between p1 and p2 ' \
                       'is greater than 0 and is statistically significant.'
    else:
        finalVerdict = 'We fail to reject the null.'

    # Function's return
    return print('\nmeanScore = ' + str(meanScore),
            '\npopulationSize = ' + str(populationSize),
            '\n\n(p1) Sample above \'' + str(meanScore) + '\' size = ' + str(aboveMeanSampleSize),
            '\n(p1) % of sample that was satisfied = ' + str(conversionAboveMean),
            '\n(p1) Standard Deviation of sample = ' + str(stDevp1),
            '\n(p2) Sample below \'' + str(meanScore) + '\' size = ' + str(belowMeanSampleSize),
            '\n(p2) % of sample that was satisfied = ' + str(conversionBelowMean),
            '\n(p2) Standard Deviation of sample = ' + str(stDevp2),
            '\n\nProportion Difference = ' + str(proportionDifference),
            '\nPooled Proportion = ' + str(pooledProportion),
            '\nStandard Error = ' + str(standardError),
            '\nZ-Statistic = ' + str(zStatistic),
            '\nP-Value = ' + str(pValue),
            '\nLower Limit = ' + str(lowerLimit),
            '\n' + finalVerdict + '\n'
            )

# A/B Test Results for 'Online Boarding'
In lines 9 through 14, we sample size, proportion and standard deviations for both samples: sample that gave above the average score and sample that gave below the average score.

In lines 20 and 21, we have a p-value of less than 5% and a lower limit greater than 0. Based off the hypothesis, Ho: p1 - p2 ≯  0 , we can reject the null and say with 95% confidence that it is statistically significant.



In [0]:
print('\nOnline boarding')
airPassSatisDF['Online boarding'].corr(airPassSatisDF['satisfaction'])
abtestfunction(airPassSatisDF['Online boarding'].values, airPassSatisDF['satisfaction'].values)


Online boarding

Ho: p1 - p2 ≯  0 
Ha: p1 - p2 > 0

meanScore = 3.2527203503054363 
populationSize = 129487 

(p1) Sample above '3.2527203503054363' size = 64294 
(p1) % of sample that was satisfied = 0.7228823840482782 
(p1) Standard Deviation of sample = 0.0017651461300519289 
(p2) Sample below '3.2527203503054363' size = 65193 
(p2) % of sample that was satisfied = 0.1500928013743807 
(p2) Standard Deviation of sample = 0.0013988308518946178 

Proportion Difference = 0.5727895826738976 
Pooled Proportion = 0.43449921613752734 
Standard Error = 0.0562492398343884 
Z-Statistic = 10.18306352868645 
P-Value = 0.0 
Lower Limit = 0.48025958314632866 
We reject the null. We can say with 95% confidence that the difference between p1 and p2 is greater than 0 and is statistically significant.



![picture](https://github.com/andrewcsmith1997/airlinePassengerSatisfaction/blob/master/Images/Analysis.PNG)

# Conclusion
In conclusion, the percentage of passengers who are overall satisfied is far greater when the sample consists of passengers who gave an above average score for a variable, online boarding, than a sample that consists of passengers who gave a below average score. 

# Recommendation
This data is useful for any airline trying to improve customer satisfaction.

I recommend that airlines use this report to see what variables drive customer satisfaction and put an emphasis on trying to improve in those areas. 

> i.e. online boarding, inflight entertainment, seat comfort,  on-board service



When customers are satisfied with these specific variables, the chances of overall satisfaction are far greater.